In [1]:
# !pip install pymp-pypi 

In [10]:
import pymp

# Create a shared array that will store the squares
with pymp.Parallel() as p:
    squares = pymp.shared.array((100,), dtype='int')
    for index in p.range(0, 100):
        # print(f"Thread {p.thread_num} processing {index}")
        squares[index] = index * index

print(squares)


[  0   1   4   9  16  25  36  49  64  81 100 121 144   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [11]:
len(squares)

100

In [16]:
import pymp

# Shared list
with pymp.Parallel() as p:
    shared_list = pymp.shared.list()

    for index in p.range(100):
        try:
            # print(f"Thread {p.thread_num} processing {index}")
            shared_list.append(index * index)
        except:
            print("Error")

print(shared_list)


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144]


In [13]:
len(shared_list)

13

In [5]:
import pymp

with pymp.Parallel() as p:
    local_sum = pymp.shared.array((p.num_threads,), dtype='int')
    for index in p.range(0, 100):
        
        local_sum[p.thread_num] += index

total_sum = sum(local_sum)
print(total_sum)

78


In [6]:
import pymp
import math

def complex_operation(input_data):
    result = math.sqrt(input_data) ** 3.5 + math.log(input_data)
    return result

input_data = range(1, 10000)  # Some large range of data

with pymp.Parallel() as p:
    results = pymp.shared.array(len(input_data))
    for index in p.range(len(input_data)):
        results[index] = complex_operation(input_data[index])

print(results)


[1.         4.05673284 7.93713346 ... 0.         0.         0.        ]


In [ ]:
import threading
from sklearn.datasets import load_iris
import numpy as np

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Define a function to calculate the mean for each species
def calculate_mean(start, end, species_id, results):
    species_data = X[y == species_id]
    results[species_id] = np.mean(species_data[start:end], axis=0)

# Create threads
results_threading = [None, None, None]
threads = []
for i in range(3):
    t = threading.Thread(target=calculate_mean, args=(0, 50, i, results_threading))
    threads.append(t)
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()

print("Threading results:", results_threading)


In [ ]:
import multiprocessing
from sklearn.datasets import load_iris
import numpy as np

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Define the worker function
def calculate_mean(start, end, species_id, return_dict):
    species_data = X[y == species_id]
    return_dict[species_id] = np.mean(species_data[start:end], axis=0)

# Create a multiprocessing manager and a dictionary to store the results
manager = multiprocessing.Manager()
return_dict = manager.dict()

# Create and start processes
processes = []
for i in range(3):
    p = multiprocessing.Process(target=calculate_mean, args=(0, 50, i, return_dict))
    processes.append(p)
    p.start()

# Wait for all processes to complete
for p in processes:
    p.join()

print("Multiprocessing results:", dict(return_dict))


In [ ]:
import pymp
from sklearn.datasets import load_iris
import numpy as np

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Prepare a shared array to store the results
results_pymp = pymp.shared.array((3, 4), dtype=np.float64)

# Parallel block
with pymp.Parallel(3) as p:
    for i in p.range(3):
        species_data = X[y == i]
        results_pymp[i] = np.mean(species_data, axis=0)

print("PyMP results:", results_pymp)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.svm import SVC
import numpy as np
import multiprocessing

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Define the worker function
def train_fold(fold_data):
    train_index, test_index = fold_data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = SVC()
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

# Prepare the folds
kf = KFold(n_splits=5)
folds = list(kf.split(X))

# Create a pool of workers and map the training function to the folds
pool = multiprocessing.Pool(processes=4)
scores = pool.map(train_fold, folds)

# Calculate the average score
average_score = np.mean(scores)
print("Average Cross-Validation Score:", average_score)
